##Installing and Setting Up Data Streaming

In [1]:
!pip install zstandard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.6 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [31]:
import zstandard as zstd
from datasets import load_dataset
pile = load_dataset("monology/pile-uncopyrighted", split="train", streaming=True)
#to continue processing after a fixed number.
pile = pile.skip(0)
#take acts like seeding and lets us choose how many samples to take and is always fixed
pile= pile.take(100000)


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [25]:
len(list(pile))

80000

In [14]:
# print(next(iter(pile)))

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

##Functionality to filter Pile-CC

In [12]:
# { 'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web ... '
#    'meta': {'pile_set_name': 'Pile-CC'}}
# use pile_set_name to only parse Pile-CC


# Functionality to pass a data sample

# for text in pile:

#   if text['meta']['pile_set_name'] == "Pile-CC":
#     text = text['text']

#   print(text)




It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.

There’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.

Concept

Working over the theme was probably one of the hardest tasks I had to face.

Originally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.

In the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge. But is it a game?



##CLD2 to identify Swahili

In [20]:
!pip install pycld2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycld2: filename=pycld2-0.41-cp310-cp310-linux_x86_64.whl size=9904036 sha256=087c0673412c064313c0ed7739137080ec544f6973c60b0270c540290d6db56e
  Stored in directory: /root/.cache/pip/wheels/be/81/31/240c89c845e008a93d98542325270007de595bfd356eb0b06c
Successfully built pycld2


In [32]:
import pandas as pd
import pycld2 as cld2
import csv
import regex

In [ ]:
def detect_language(text):
    try:

        return cld2.detect(text)[2][0][0]  # the detected language (can change this to include Swahili if its the second most confident as well)
    except Exception as e:
        print(f"Error: {e}")
        return None
#FOR Storing in a csv file:

# def process_texts(texts, output_csv):
#     with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(['Text'])
#         for text in texts:
#             if detect_language(text) == 'SWAHILI':
#                 writer.writerow([text])
#     print(f"Stored in {output_csv}")

#FOR Storing in a text file and checking if the first 5 words of the text file only.


RE_BAD_CHARS = regex.compile(r"\p{Cc}|\p{Cs}")

def remove_bad_chars(text):
    return RE_BAD_CHARS.sub("", text)



def process_texts(texts, output_txt):
    with open(output_txt, mode='w', encoding='utf-8') as txt_file:
        for text in texts:
            if detect_language(' '.join(remove_bad_chars(text).split()[:10])) == 'SWAHILI':
                txt_file.write(text + '\n')
                print(f"Stored in {output_txt}")

# Load in the texts by streaming data from here.(currently takes in an array.)
# texts = [
#     "Hii ni mfano wa maandishi ya Kiswahili.",
#     "This is an example of English text.",
#     "Huyu ni rafiki yangu.",
#     "Voici un exemple de texte en français."
# ]

#if ur not limited to a particular sub domain of pile then dont pass if check pass  process_texts(text, 'swahili_pile_cc.txt') directly
for text in pile:

  if text['meta']['pile_set_name'] == "Pile-CC":
    text = text['text']
    process_texts(text, 'swahili_pile_cc.txt')
#   print(text)
# process_texts(texts, 'swahili_pile_cc.txt')